In [1]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName("Missing").getOrCreate()

22/01/19 04:58:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/01/19 04:58:53 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [7]:
train = spark.read.csv("test4.csv", header=True, inferSchema=True)
train.show()

+-------+---+----------+------+
|   Name|age|experience|Salary|
+-------+---+----------+------+
|  danny| 30|        10| 30000|
|hyunjoo| 33|      null| 40000|
| eileen| 23|         5| 70000|
| hailey| 44|         4|  3400|
|   john| 22|         3| 40000|
|    kim| 36|         6|200000|
|   kim2| 87|         7| 30000|
|   lora| 49|        15|152020|
+-------+---+----------+------+



In [8]:
train.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [9]:
train.columns

['Name', 'age', 'experience', 'Salary']

In [11]:

train.dtypes

[('Name', 'string'), ('age', 'int'), ('experience', 'int'), ('Salary', 'int')]

[age, experience] --->new feature ----> indenpendent feature

In [27]:
train.show()

+-------+---+----------+------+
|   Name|age|experience|Salary|
+-------+---+----------+------+
|  danny| 30|        10| 30000|
|hyunjoo| 33|      null| 40000|
| eileen| 23|         5| 70000|
| hailey| 44|         4|  3400|
|   john| 22|         3| 40000|
|    kim| 36|         6|200000|
|   kim2| 87|         7| 30000|
|   lora| 49|        15|152020|
+-------+---+----------+------+



In [33]:
from pyspark.ml.feature import Imputer
imputer = Imputer(inputCol='experience',
                  outputCol='experience_imputed').setStrategy("mean")

In [46]:
df = imputer.fit(train).transform(train)
df.show()

+-------+---+----------+------+------------------+
|   Name|age|experience|Salary|experience_imputed|
+-------+---+----------+------+------------------+
|  danny| 30|        10| 30000|                10|
|hyunjoo| 33|      null| 40000|                 7|
| eileen| 23|         5| 70000|                 5|
| hailey| 44|         4|  3400|                 4|
|   john| 22|         3| 40000|                 3|
|    kim| 36|         6|200000|                 6|
|   kim2| 87|         7| 30000|                 7|
|   lora| 49|        15|152020|                15|
+-------+---+----------+------+------------------+



In [47]:
df =df.drop("experience")
df.show()

+-------+---+------+------------------+
|   Name|age|Salary|experience_imputed|
+-------+---+------+------------------+
|  danny| 30| 30000|                10|
|hyunjoo| 33| 40000|                 7|
| eileen| 23| 70000|                 5|
| hailey| 44|  3400|                 4|
|   john| 22| 40000|                 3|
|    kim| 36|200000|                 6|
|   kim2| 87| 30000|                 7|
|   lora| 49|152020|                15|
+-------+---+------+------------------+



In [54]:
from pyspark.ml.feature import VectorAssembler
featureassember = VectorAssembler(inputCols=["age","experience_imputed"],
                                outputCol="Independent")

In [55]:
featureassember

VectorAssembler_8e7796f2b37f

In [56]:
output = featureassember.transform(df)
output.show()

+-------+---+------+------------------+-----------+
|   Name|age|Salary|experience_imputed|Independent|
+-------+---+------+------------------+-----------+
|  danny| 30| 30000|                10|[30.0,10.0]|
|hyunjoo| 33| 40000|                 7| [33.0,7.0]|
| eileen| 23| 70000|                 5| [23.0,5.0]|
| hailey| 44|  3400|                 4| [44.0,4.0]|
|   john| 22| 40000|                 3| [22.0,3.0]|
|    kim| 36|200000|                 6| [36.0,6.0]|
|   kim2| 87| 30000|                 7| [87.0,7.0]|
|   lora| 49|152020|                15|[49.0,15.0]|
+-------+---+------+------------------+-----------+



In [58]:

output.columns

['Name', 'age', 'Salary', 'experience_imputed', 'Independent']

In [59]:
finalized_df = output.select(["Independent","Salary"])
finalized_df.show()

+-----------+------+
|Independent|Salary|
+-----------+------+
|[30.0,10.0]| 30000|
| [33.0,7.0]| 40000|
| [23.0,5.0]| 70000|
| [44.0,4.0]|  3400|
| [22.0,3.0]| 40000|
| [36.0,6.0]|200000|
| [87.0,7.0]| 30000|
|[49.0,15.0]|152020|
+-----------+------+



In [60]:
from pyspark.ml.regression import LinearRegression

In [62]:
train_df, test_df = finalized_df.randomSplit([0.75,0.25])
regressor = LinearRegression(featuresCol = "Independent", labelCol="Salary")
regressor=regressor.fit(train_df)

22/01/19 05:29:35 WARN Instrumentation: [1f616092] regParam is zero, which might cause numerical instability and overfitting.
22/01/19 05:29:35 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
22/01/19 05:29:35 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
22/01/19 05:29:35 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
22/01/19 05:29:35 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


In [63]:
regressor

LinearRegressionModel: uid=LinearRegression_d36087d983e1, numFeatures=2

In [65]:
regressor.coefficients

DenseVector([889.2075, 5731.4454])

In [66]:
regressor.intercept

5443.649715996512

In [67]:
pred = regressor.evaluate(test_df)
pred

In [68]:
pred.predictions.show()

+-----------+------+-----------------+
|Independent|Salary|       prediction|
+-----------+------+-----------------+
| [87.0,7.0]| 30000|122924.8212102104|
+-----------+------+-----------------+



In [69]:
df.show()

+-------+---+------+------------------+
|   Name|age|Salary|experience_imputed|
+-------+---+------+------------------+
|  danny| 30| 30000|                10|
|hyunjoo| 33| 40000|                 7|
| eileen| 23| 70000|                 5|
| hailey| 44|  3400|                 4|
|   john| 22| 40000|                 3|
|    kim| 36|200000|                 6|
|   kim2| 87| 30000|                 7|
|   lora| 49|152020|                15|
+-------+---+------+------------------+



In [70]:
pred.meanAbsoluteError

92924.8212102104

In [71]:
pred.meanSquaredError

8635022396.94957